## Topic Modelling(Text Analytics)

* Whether the reviews are talking about the product or platform ?
* Top 5 features about the PRODUCT that the customers are talking about(across brands).
* Sentiments attached with the features to be combined into "Others"
* If we can measure the regression coefficients or correlation between the pos/neg sentiments about a feature and the rating for the product

In [21]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

import re
import numpy as np
import pandas as pd
import nltk

# Plotting tools
# import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [120]:
# Visualizaing pyLDA model
from pyLDAvis import sklearn as sklearn_lda
import pickle 
import pyLDAvis

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manish.singh04\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
data = pd.read_excel("Hanger Reviews consolidated.xlsx") # read data

In [4]:
print(data.shape)
data[["Product","Review", "Rating"]].head()

(4077, 17)


,Product,Review,Rating
0,3M Command Picture Hanging,This is a review of the Command Picture Hangin...,5
1,3M Command Picture Hanging,Cleaned area with alcohol and followed steps e...,1
2,3M Command Picture Hanging,Update: Feb 2018. It has been a little over a ...,4
3,3M Command Picture Hanging,"Do not buy this from Amazon. Go to your local,...",1
4,3M Command Picture Hanging,I have found about a million uses for these th...,5


In [7]:
data['Product'].value_counts()

3M Command Picture Hanging    2331
VELCRO                         744
Hangman Products               435
Beehive Picture Hangers        266
GoodHangups                    191
eBoot                          110
Name: Product, dtype: int64

In [8]:
#sample review
data.Review[10]

"Fabulous....a bit difficult to get up, initially. You *need* to level your frame at the bottom then angle it up to place the tabs, and they're serious about letting those tabs sit for at least an hour before you hang the heavier pictures (learn from my fail). If you follow the instructions, they work *brilliantly*"

In [85]:
# Helper Functions
# Preprocessing

# NLTK Stop words
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Define functions for lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(sent) 
        texts_out.extend([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        
    return " ".join(texts_out)

nlp = spacy.load('en', disable=['parser', 'ner'])

def pos_tagger(text,)

In [66]:
# Remove punctuation
data['Review_processed'] = data['Review'].apply(lambda x: re.sub('[\W\s]', ' ', x.lower()))

In [71]:
# Remove Stopwords and numbers
data['Review_processed'] = data['Review_processed'].str.replace('\d+', '')
data['Review_processed'] = data['Review_processed'].apply(lambda x: [item for item in x.split() if item not in stop_words])

In [82]:
print(data['Review_processed'][0])

['review', 'command', 'picture', 'hanging', 'strips', 'value', 'pack', 'large', 'white', 'strips', 'get', 'total', 'individual', 'strips', 'box', 'two', 'strips', 'minimum', 'needed', 'hang', 'object', 'summary', 'daughter', 'part', 'local', 'roller', 'skating', 'club', 'lucky', 'oversee', 'plaque', 'wall', 'rink', 'permit', 'nails', 'wall', 'find', 'ways', 'mount', 'items', 'without', 'damaging', 'wall', 'served', 'perfect', 'solution', 'us', 'noted', 'description', 'truly', 'offer', 'damage', 'free', 'hanging', 'items', 'tab', 'one', 'end', 'lift', 'need', 'remove', 'strip', 'surface', 'constantly', 'moving', 'items', 'around', 'let', 'us', 'ease', 'speed', 'leaving', 'damage', 'strip', 'measures', 'long', '¾', 'wide', 'excluding', 'tab', 'remove', 'strip', 'rated', 'hold', 'lbs', 'using', 'two', 'sets', 'item', 'give', 'ability', 'hold', 'lbs', 'using', 'four', 'sets', 'would', 'hold', 'lbs', 'holding', 'light', 'weight', 'frames', 'heavy', 'wood', 'plaques', 'ease', 'tips', 'clean'

In [87]:
# lemmatization
data['Review_processed'] = data['Review_processed'].apply(lemmatization)

In [89]:
data['Review_processed'][0]

'review command picture hang strip value pack large white strip get total individual strip box strip minimum need hang object summary daughter part local roller skate club lucky oversee plaque wall rink permit nail wall find way mount item damage wall serve perfect solution note description truly offer damage free hang item tab end lift nee remove strip surface constantly move item around let ease speed leave damage strip measure long wide exclude tab remove strip rate hold lb use set item give ability hold lb use set would hold lb hold light weight frame heavy wood plaque ease tip clean surface use good result instal press hold approx second remove item wall twist pull item try pull straight review develop issue ongoing update review accordingly'

In [91]:
# Load the library with the CountVectorizer method
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation as LDA

In [92]:
# Initialise the count vectorizer
count_vectorizer = CountVectorizer()

In [93]:
# Fit and transform the processed Review
count_data = count_vectorizer.fit_transform(data.Review_processed)

In [94]:
count_data

<4077x3595 sparse matrix of type '<class 'numpy.int64'>'
	with 58279 stored elements in Compressed Sparse Row format>

In [123]:
# Set parameters
num_topics = 4
batch_size = 500 
passes = 20 
iterations = 400
eval_every = 1  

In [124]:
# Create and fit the LDA model
lda = LDA(n_components = num_topics, 
          batch_size = batch_size, 
          max_iter = iterations,
          evaluate_every = eval_every,
         random_state = 101)
lda.fit(count_data)

LatentDirichletAllocation(batch_size=500, doc_topic_prior=None,
             evaluate_every=1, learning_decay=0.7, learning_method='batch',
             learning_offset=10.0, max_doc_update_iter=100, max_iter=400,
             mean_change_tol=0.001, n_components=4, n_jobs=None,
             n_topics=None, perp_tol=0.1, random_state=101,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [125]:
# Function
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(",".join([words[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [126]:
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, 10)

Topics found via LDA:

Topic #0:
great,work,product,good,price,velcro,expect,need,easy,time

Topic #1:
hang,wall,picture,easy,mirror,frame,hanger,use,make,work

Topic #2:
wall,strip,picture,hang,frame,use,fall,command,hold,product

Topic #3:
hold,velcro,use,stick,poster,work,board,side,back,strong


In [127]:
LDAvis_prepared = sklearn_lda.prepare(lda, count_data, count_vectorizer)

C:\Users\manish.singh04\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [128]:
pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(num_topics) +'.html')

In [141]:
# Sentiments Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [152]:
def polarity_score(sentence):
    score = analyser.polarity_scores(sentence)
    print(score)
#     print("{:-<40} {}".format(sentence, str(score)))

In [153]:
data['Review'].apply(polarity_score)

{'neg': 0.021, 'neu': 0.859, 'pos': 0.12, 'compound': 0.9747}
{'neg': 0.091, 'neu': 0.819, 'pos': 0.091, 'compound': 0.4527}
{'neg': 0.08, 'neu': 0.837, 'pos': 0.083, 'compound': 0.273}
{'neg': 0.054, 'neu': 0.875, 'pos': 0.071, 'compound': 0.4951}
{'neg': 0.0, 'neu': 0.915, 'pos': 0.085, 'compound': 0.8745}
{'neg': 0.023, 'neu': 0.824, 'pos': 0.153, 'compound': 0.9365}
{'neg': 0.108, 'neu': 0.764, 'pos': 0.129, 'compound': 0.5086}
{'neg': 0.0, 'neu': 0.687, 'pos': 0.313, 'compound': 0.6249}
{'neg': 0.041, 'neu': 0.959, 'pos': 0.0, 'compound': -0.6486}
{'neg': 0.101, 'neu': 0.879, 'pos': 0.019, 'compound': -0.8443}
{'neg': 0.067, 'neu': 0.933, 'pos': 0.0, 'compound': -0.4215}
{'neg': 0.059, 'neu': 0.85, 'pos': 0.09, 'compound': 0.4635}
{'neg': 0.032, 'neu': 0.868, 'pos': 0.1, 'compound': 0.9651}
{'neg': 0.173, 'neu': 0.684, 'pos': 0.143, 'compound': -0.3557}
{'neg': 0.027, 'neu': 0.928, 'pos': 0.044, 'compound': 0.6209}
{'neg': 0.055, 'neu': 0.898, 'pos': 0.047, 'compound': 0.2213}
{'n

{'neg': 0.0, 'neu': 0.852, 'pos': 0.148, 'compound': 0.9042}
{'neg': 0.059, 'neu': 0.847, 'pos': 0.094, 'compound': 0.3428}
{'neg': 0.0, 'neu': 0.678, 'pos': 0.322, 'compound': 0.9635}
{'neg': 0.052, 'neu': 0.841, 'pos': 0.108, 'compound': 0.3818}
{'neg': 0.068, 'neu': 0.825, 'pos': 0.107, 'compound': 0.6916}
{'neg': 0.057, 'neu': 0.909, 'pos': 0.033, 'compound': -0.128}
{'neg': 0.0, 'neu': 0.828, 'pos': 0.172, 'compound': 0.7888}
{'neg': 0.0, 'neu': 0.805, 'pos': 0.195, 'compound': 0.9756}
{'neg': 0.032, 'neu': 0.739, 'pos': 0.229, 'compound': 0.8064}
{'neg': 0.0, 'neu': 0.834, 'pos': 0.166, 'compound': 0.7163}
{'neg': 0.1, 'neu': 0.854, 'pos': 0.047, 'compound': -0.4082}
{'neg': 0.077, 'neu': 0.688, 'pos': 0.235, 'compound': 0.8214}
{'neg': 0.0, 'neu': 0.875, 'pos': 0.125, 'compound': 0.7713}
{'neg': 0.0, 'neu': 0.829, 'pos': 0.171, 'compound': 0.7783}
{'neg': 0.041, 'neu': 0.792, 'pos': 0.167, 'compound': 0.7096}
{'neg': 0.03, 'neu': 0.847, 'pos': 0.123, 'compound': 0.7384}
{'neg': 

{'neg': 0.0, 'neu': 0.711, 'pos': 0.289, 'compound': 0.8796}
{'neg': 0.041, 'neu': 0.885, 'pos': 0.074, 'compound': 0.5563}
{'neg': 0.094, 'neu': 0.906, 'pos': 0.0, 'compound': -0.4215}
{'neg': 0.0, 'neu': 0.843, 'pos': 0.157, 'compound': 0.6449}
{'neg': 0.037, 'neu': 0.748, 'pos': 0.215, 'compound': 0.7883}
{'neg': 0.0, 'neu': 0.781, 'pos': 0.219, 'compound': 0.6369}
{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}
{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4404}
{'neg': 0.144, 'neu': 0.645, 'pos': 0.211, 'compound': 0.0}
{'neg': 0.066, 'neu': 0.775, 'pos': 0.16, 'compound': 0.6331}
{'neg': 0.0, 'neu': 0.439, 'pos': 0.561, 'compound': 0.9764}
{'neg': 0.072, 'neu': 0.827, 'pos': 0.102, 'compound': 0.4029}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.038, 'neu': 0.887, 'pos': 0.075, 'compound': 0.2732}
{'neg': 0.0, 'neu': 0.733, 'pos': 0.267, 'compound': 0.6556}
{'neg': 0.109, 'neu': 0.82, 'pos': 0.071, 'compound': -0.5722}
{'neg': 0.062, 'neu': 0.

{'neg': 0.108, 'neu': 0.892, 'pos': 0.0, 'compound': -0.2057}
{'neg': 0.219, 'neu': 0.725, 'pos': 0.055, 'compound': -0.8225}
{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.5448}
{'neg': 0.0, 'neu': 0.87, 'pos': 0.13, 'compound': 0.3384}
{'neg': 0.04, 'neu': 0.83, 'pos': 0.13, 'compound': 0.5868}
{'neg': 0.0, 'neu': 0.637, 'pos': 0.363, 'compound': 0.6908}
{'neg': 0.0, 'neu': 0.682, 'pos': 0.318, 'compound': 0.6369}
{'neg': 0.0, 'neu': 0.879, 'pos': 0.121, 'compound': 0.5719}
{'neg': 0.0, 'neu': 0.701, 'pos': 0.299, 'compound': 0.8687}
{'neg': 0.0, 'neu': 0.715, 'pos': 0.285, 'compound': 0.8475}
{'neg': 0.0, 'neu': 0.674, 'pos': 0.326, 'compound': 0.9022}
{'neg': 0.0, 'neu': 0.598, 'pos': 0.402, 'compound': 0.8668}
{'neg': 0.107, 'neu': 0.893, 'pos': 0.0, 'compound': -0.7096}
{'neg': 0.049, 'neu': 0.951, 'pos': 0.0, 'compound': -0.1531}
{'neg': 0.043, 'neu': 0.887, 'pos': 0.069, 'compound': 0.296}
{'neg': 0.0, 'neu': 0.734, 'pos': 0.266, 'compound': 0.4404}
{'neg': 0.0, 'neu': 

{'neg': 0.045, 'neu': 0.955, 'pos': 0.0, 'compound': -0.4019}
{'neg': 0.0, 'neu': 0.813, 'pos': 0.187, 'compound': 0.8689}
{'neg': 0.03, 'neu': 0.855, 'pos': 0.116, 'compound': 0.7606}
{'neg': 0.0, 'neu': 0.724, 'pos': 0.276, 'compound': 0.6369}
{'neg': 0.0, 'neu': 0.786, 'pos': 0.214, 'compound': 0.6556}
{'neg': 0.093, 'neu': 0.907, 'pos': 0.0, 'compound': -0.2405}
{'neg': 0.076, 'neu': 0.76, 'pos': 0.164, 'compound': 0.3612}
{'neg': 0.0, 'neu': 0.682, 'pos': 0.318, 'compound': 0.6369}
{'neg': 0.0, 'neu': 0.816, 'pos': 0.184, 'compound': 0.7163}
{'neg': 0.093, 'neu': 0.826, 'pos': 0.08, 'compound': 0.2486}
{'neg': 0.105, 'neu': 0.895, 'pos': 0.0, 'compound': -0.3786}
{'neg': 0.274, 'neu': 0.406, 'pos': 0.32, 'compound': 0.2263}
{'neg': 0.146, 'neu': 0.738, 'pos': 0.116, 'compound': -0.296}
{'neg': 0.0, 'neu': 0.803, 'pos': 0.197, 'compound': 0.4019}
{'neg': 0.057, 'neu': 0.733, 'pos': 0.21, 'compound': 0.9349}
{'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound': 0.5777}
{'neg': 0.0, 'neu'

...

{'neg': 0.086, 'neu': 0.758, 'pos': 0.155, 'compound': 0.6734}
{'neg': 0.0, 'neu': 0.63, 'pos': 0.37, 'compound': 0.9067}
{'neg': 0.0, 'neu': 0.609, 'pos': 0.391, 'compound': 0.723}
{'neg': 0.0, 'neu': 0.708, 'pos': 0.292, 'compound': 0.7482}
{'neg': 0.0, 'neu': 0.874, 'pos': 0.126, 'compound': 0.5574}
{'neg': 0.038, 'neu': 0.644, 'pos': 0.319, 'compound': 0.9407}
{'neg': 0.0, 'neu': 0.791, 'pos': 0.209, 'compound': 0.9129}
{'neg': 0.0, 'neu': 0.641, 'pos': 0.359, 'compound': 0.4215}
{'neg': 0.0, 'neu': 0.609, 'pos': 0.391, 'compound': 0.723}
{'neg': 0.0, 'neu': 0.708, 'pos': 0.292, 'compound': 0.7482}
{'neg': 0.0, 'neu': 0.874, 'pos': 0.126, 'compound': 0.5574}
{'neg': 0.038, 'neu': 0.644, 'pos': 0.319, 'compound': 0.9407}
{'neg': 0.0, 'neu': 0.791, 'pos': 0.209, 'compound': 0.9129}
{'neg': 0.0, 'neu': 0.641, 'pos': 0.359, 'compound': 0.4215}
{'neg': 0.027, 'neu': 0.726, 'pos': 0.247, 'compound': 0.9323}
{'neg': 0.0, 'neu': 0.468, 'pos': 0.532, 'compound': 0.8812}
{'neg': 0.0, 'neu': 

{'neg': 0.0, 'neu': 0.608, 'pos': 0.392, 'compound': 0.7959}
{'neg': 0.0, 'neu': 0.766, 'pos': 0.234, 'compound': 0.8852}
{'neg': 0.0, 'neu': 0.62, 'pos': 0.38, 'compound': 0.6468}
{'neg': 0.036, 'neu': 0.687, 'pos': 0.278, 'compound': 0.9683}
{'neg': 0.076, 'neu': 0.858, 'pos': 0.066, 'compound': -0.3457}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.622, 'neu': 0.378, 'pos': 0.0, 'compound': -0.5096}
{'neg': 0.044, 'neu': 0.778, 'pos': 0.179, 'compound': 0.9656}
{'neg': 0.065, 'neu': 0.825, 'pos': 0.111, 'compound': 0.8762}
{'neg': 0.0, 'neu': 0.653, 'pos': 0.347, 'compound': 0.4927}
{'neg': 0.0, 'neu': 0.958, 'pos': 0.042, 'compound': 0.4359}
{'neg': 0.03, 'neu': 0.941, 'pos': 0.028, 'compound': 0.1901}
{'neg': 0.114, 'neu': 0.79, 'pos': 0.096, 'compound': -0.2915}
{'neg': 0.04, 'neu': 0.821, 'pos': 0.139, 'compound': 0.929}
{'neg': 0.138, 'neu': 0.766, 'pos': 0.095, 'compound': -0.6557}
{'neg': 0.015, 'neu': 0.853, 'pos': 0.132, 'compound': 0.9761}
{'neg': 0.106, 

{'neg': 0.0, 'neu': 0.808, 'pos': 0.192, 'compound': 0.8735}
{'neg': 0.082, 'neu': 0.836, 'pos': 0.082, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.826, 'pos': 0.174, 'compound': 0.7736}
{'neg': 0.047, 'neu': 0.871, 'pos': 0.082, 'compound': 0.2495}
{'neg': 0.066, 'neu': 0.757, 'pos': 0.177, 'compound': 0.68}
{'neg': 0.0, 'neu': 0.427, 'pos': 0.573, 'compound': 0.3989}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.062, 'neu': 0.877, 'pos': 0.062, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.778, 'pos': 0.222, 'compound': 0.9207}
{'neg': 0.16, 'neu': 0.639, 'pos': 0.201, 'compound': 0.2247}
{'neg': 0.0, 'neu': 0.655, 'pos': 0.345, 'compound': 0.8591}
{'neg': 0.0, 'neu': 0.757, 'pos': 0.243, 'compound': 0.8641}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.782, 'pos': 0.218, 'compound': 0.5983}
{'neg': 0.098, 'neu': 0.661, 'pos': 0.241, 'compound': 0.659}
{'neg': 0.131, 'neu': 0.869, 'pos': 0.0, 'compound': -0.2411}
{'neg': 0.105, 'neu': 0.736, 'pos':

{'neg': 0.089, 'neu': 0.661, 'pos': 0.25, 'compound': 0.9484}
{'neg': 0.169, 'neu': 0.805, 'pos': 0.026, 'compound': -0.9298}
{'neg': 0.0, 'neu': 0.876, 'pos': 0.124, 'compound': 0.4005}
{'neg': 0.205, 'neu': 0.795, 'pos': 0.0, 'compound': -0.4767}
{'neg': 0.0, 'neu': 0.3, 'pos': 0.7, 'compound': 0.7906}
{'neg': 0.051, 'neu': 0.795, 'pos': 0.154, 'compound': 0.7355}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.127, 'neu': 0.78, 'pos': 0.094, 'compound': -0.3417}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.192, 'neu': 0.718, 'pos': 0.09, 'compound': -0.8826}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.313, 'pos': 0.687, 'compound': 0.6588}
{'neg': 0.0, 'neu': 0.413, 'pos': 0.587, 'compound': 0.7964}
{'neg': 0.063, 'neu': 0.659, 'pos': 0.278, 'compound': 0.9771}
{'neg': 0.0, 'neu': 0.73, 'pos': 0.27, 'compound': 0.3802}
{'neg': 0.0, 'neu': 0.885, 'pos': 0.115, 'compound': 0.2382}
{'neg': 0.156, 'neu': 0.72, 'pos': 0.1

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
4047    None
4048    None
4049    None
4050    None
4051    None
4052    None
4053    None
4054    None
4055    None
4056    None
4057    None
4058    None
4059    None
4060    None
4061    None
4062    None
4063    None
4064    None
4065    None
4066    None
4067    None
4068    None
4069    None
4070    None
4071    None
4072    None
4073    None
4074    None
4075    None
4076    None
Name: Review, Length: 4077, dtype: object

In [155]:
data['Review']

0       This is a review of the Command Picture Hangin...
1       Cleaned area with alcohol and followed steps e...
2       Update: Feb 2018. It has been a little over a ...
3       Do not buy this from Amazon. Go to your local,...
4       I have found about a million uses for these th...
5       These 3M Command strips are miraculous. They t...
6       I purchased the "Command™ Large Picture Hangin...
7       These are great for hanging pictures, painting...
8       These are NOT official 3M Command strips, but ...
9       This product was awful...when I tried to peel ...
10      Fabulous....a bit difficult to get up, initial...
11      DO NOT BUY! DO NOT BUY!! I had them in use for...
12      I've used these for a very long time and I cou...
13      I love these things. I did a wall collage that...
14      I purchased these for a piece of art I wanted ...
15      I was expecting generic velcro.  But, these st...
16      Reliable.I use these to put things up to about...
17      As an 